In [2]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import joblib

# Load ratings data
ratings = pd.read_csv('rating.csv')

# Filter out users who haven't rated many animes to reduce noise
ratings_filtered = ratings.groupby('user_id').filter(lambda x: len(x) >= 10)

# The Reader class is used to parse a file containing ratings
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order)
data = Dataset.load_from_df(ratings_filtered[['user_id', 'anime_id', 'rating']], reader)

# Split data into training and test set
trainset, testset = train_test_split(data, test_size=.25)

# Use the SVD algorithm.
algo = SVD(n_epochs=10)

# Train the algorithm on the trainset
algo.fit(trainset)

# Save the model
joblib.dump(algo, 'anime_model.pkl')


['anime_model.pkl']

In [10]:
import joblib
# Load your trained model
model = joblib.load('anime_model.pkl')

# Load your anime data
anime = pd.read_csv('anime.csv')

def recommend_animes(model, liked_anime_ids, n_recommendations):
    # Create a pseudo-user who likes the specified animes
    pseudo_user_id = 'pseudo_user'

    # Get a list of all anime IDs
    all_anime_ids = anime['anime_id'].unique()

    # Predict ratings for all animes that the pseudo-user hasn't seen yet
    predictions = [(anime_id, model.predict(pseudo_user_id, anime_id).est) for anime_id in all_anime_ids if anime_id not in liked_anime_ids]

    # Sort by predicted rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get the top n_recommendations
    top_n = [anime_id for anime_id, _ in predictions[:n_recommendations]]

    return top_n

def filter_by_genre(recommended_anime_ids, liked_anime_ids):
    # Get the genres of the liked animes
    liked_genres = set()
    for genres in anime.loc[anime['anime_id'].isin(liked_anime_ids), 'genre']:
        liked_genres.update(genres.split(', '))
    print(f"Liked genres: {liked_genres}")

    # Filter the recommended animes based on these genres
    filtered_anime_ids = []
    for id in recommended_anime_ids:
        anime_genres = anime.loc[anime['anime_id'] == id, 'genre'].item().split(', ')
        print(f"Anime ID {id} genres: {anime_genres}")
        if any(genre in liked_genres for genre in anime_genres):
            filtered_anime_ids.append(id)
    
    return filtered_anime_ids



# Create a dictionary to map anime names to IDs
anime_dict = pd.Series(anime.anime_id.values,index=anime.name).to_dict()

# Create a list of anime names
anime_list = list(anime.name.values)

# Create dropdown selectors for the user to choose their favorite animes
selected_anime_1 = "Cross Game"
selected_anime_2 = "Bakuman. 2nd Season"
selected_anime_3 = "Nichijou"

    # Map from anime names to IDs
anime_ids = [anime_dict[selected_anime_1], anime_dict[selected_anime_2], anime_dict[selected_anime_3]]
    
    # Generate recommendations
recommended_anime_ids = recommend_animes(model, anime_ids, n_recommendations=10)
print(recommended_anime_ids)
recommended_anime_ids = filter_by_genre(recommended_anime_ids, anime_ids)
print(recommended_anime_ids)
    # Map from anime IDs back to names
recommended_anime_names = anime[anime['anime_id'].isin(recommended_anime_ids)]['name'].tolist()

    
    # Display the recommended animes
print(recommended_anime_names)


[820, 28977, 918, 11061, 5420, 5114, 9253, 32281, 4181, 9969]
Liked genres: {'Sports', 'Romance', 'Slice of Life', 'Shounen', 'School', 'Drama', 'Comedy'}
Anime ID 820 genres: ['Drama', 'Military', 'Sci-Fi', 'Space']
Anime ID 28977 genres: ['Action', 'Comedy', 'Historical', 'Parody', 'Samurai', 'Sci-Fi', 'Shounen']
Anime ID 918 genres: ['Action', 'Comedy', 'Historical', 'Parody', 'Samurai', 'Sci-Fi', 'Shounen']
Anime ID 11061 genres: ['Action', 'Adventure', 'Shounen', 'Super Power']
Anime ID 5420 genres: ['Drama', 'Fantasy']
Anime ID 5114 genres: ['Action', 'Adventure', 'Drama', 'Fantasy', 'Magic', 'Military', 'Shounen']
Anime ID 9253 genres: ['Sci-Fi', 'Thriller']
Anime ID 32281 genres: ['Drama', 'Romance', 'School', 'Supernatural']
Anime ID 4181 genres: ['Drama', 'Fantasy', 'Romance', 'Slice of Life', 'Supernatural']
Anime ID 9969 genres: ['Action', 'Comedy', 'Historical', 'Parody', 'Samurai', 'Sci-Fi', 'Shounen']
[820, 28977, 918, 11061, 5420, 5114, 32281, 4181, 9969]
['Kimi no Na w